In [9]:

import os
from time import sleep

import polars as pl
import requests
from tqdm import tqdm


In [2]:
cc_data = "https://data.commoncrawl.org/"


In [3]:
with open('cc-index-table.paths', 'r') as file:
    index_table_paths = file.read().splitlines()
index_lst = [index for index in index_table_paths]
# index_lst = index_lst[600:]
# index_lst = index_lst[:300]
# index_lst = index_lst[600:]

In [7]:
test_file = cc_data + index_lst[0]
print(test_file)
test_index_gz = requests.get(test_file, stream=True)
test_file_name = 'data/iter_index_table_test'
with open(test_file_name, 'wb') as kk:
    kk.write(test_index_gz.content)
# test_index_gz.text[:100]

https://data.commoncrawl.org/cc-index/table/cc-main/warc/crawl=CC-MAIN-2025-05/subset=crawldiagnostics/part-00000-88b30a59-3c73-48ba-a167-077611bfd245.c000.gz.parquet


In [12]:
test_table = pl.read_parquet(test_file_name).select(['url_host_tld', 'fetch_status', 'warc_filename', 'url', 'url_path',
                                                     'url_host_registered_domain'])
test_table.head()

url_host_tld,fetch_status,warc_filename,url,url_path,url_host_registered_domain
str,i16,str,str,str,str
"""ru""",304,"""crawl-data/CC-MAIN-2025-05/seg…","""https://ya5111.ru/2474.html""","""/2474.html""","""ya5111.ru"""
"""ru""",304,"""crawl-data/CC-MAIN-2025-05/seg…","""https://ya5111.ru/3874.html""","""/3874.html""","""ya5111.ru"""
"""ru""",304,"""crawl-data/CC-MAIN-2025-05/seg…","""https://ya5111.ru/5114.html""","""/5114.html""","""ya5111.ru"""
"""ru""",304,"""crawl-data/CC-MAIN-2025-05/seg…","""https://ya5111.ru/5135.html""","""/5135.html""","""ya5111.ru"""
"""ru""",304,"""crawl-data/CC-MAIN-2025-05/seg…","""https://ya5111.ru/5140.html""","""/5140.html""","""ya5111.ru"""


In [5]:
print(cc_data + index_lst[1])

https://data.commoncrawl.org/cc-index/table/cc-main/warc/crawl=CC-MAIN-2022-33/subset=warc/part-00001-d466b69e-be2b-4525-ac34-1b10d57329da.c000.gz.parquet


In [38]:
progress = list()


In [41]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36 Edg/103.0.1264.77"
}
domain='com'
for index in (pbar := tqdm(index_lst[:10])): #printing which file is currently processed
    if index in progress: # so it won't start from the beginning of the list, if an error occurred, saving time
        continue

    URL = cc_data + index
    file_name = index.split('/')[-1]
    pbar.set_description(file_name)
    # sleep(0.5)

    index_gz = requests.get(URL, stream=True, headers=headers
                            #verify=False,
                            )

    # print(index_gz)
    # print(URL)
    # print(index)
    sleep(1.5)
    with open(f'data/{file_name}', 'wb') as ff:
        ff.write(index_gz.content)
    ff.close()
    sleep(1)

    table = pl.read_parquet(f'data/{file_name}').select(
        ['url_host_tld', 'fetch_status', 'warc_filename', 'url', 'url_path',
         'url_host_registered_domain']).filter(
        (pl.col('url_host_tld') == domain) &
         (pl.col('fetch_status') != 404)
    )
    # print(table.shape[0])
    # print(table.head())

    if table.shape[0] > 1: # table has at least 1 row
        #table.to_csv(f'{file_name}.csv', index=False)

        print(f'{domain} found in {file_name}')
        continue #leave the downloaded .parquet file on disk fo further use


    progress.append(index)
    sleep(1)
    os.remove(f'data/{file_name}')


part-00002-88b30a59-3c73-48ba-a167-077611bfd245.c000.gz.parquet:  20%|██        | 2/10 [01:00<03:33, 26.71s/it]

com found in part-00001-88b30a59-3c73-48ba-a167-077611bfd245.c000.gz.parquet


part-00004-88b30a59-3c73-48ba-a167-077611bfd245.c000.gz.parquet:  40%|████      | 4/10 [01:43<02:19, 23.23s/it]

com found in part-00003-88b30a59-3c73-48ba-a167-077611bfd245.c000.gz.parquet


part-00009-88b30a59-3c73-48ba-a167-077611bfd245.c000.gz.parquet:  90%|█████████ | 9/10 [03:48<00:25, 25.36s/it]

com found in part-00008-88b30a59-3c73-48ba-a167-077611bfd245.c000.gz.parquet


part-00009-88b30a59-3c73-48ba-a167-077611bfd245.c000.gz.parquet: 100%|██████████| 10/10 [04:11<00:00, 25.19s/it]


In [42]:
table = pl.read_parquet('data/part-00001-88b30a59-3c73-48ba-a167-077611bfd245.c000.gz.parquet')
table.shape[0]

761188

In [43]:
table.head(25).select(
    ['url_host_tld', 'fetch_status', 'warc_filename', 'url', 'url_path',
     'url_host_registered_domain'])

url_host_tld,fetch_status,warc_filename,url,url_path,url_host_registered_domain
str,i16,str,str,str,str
"""com""",301,"""crawl-data/CC-MAIN-2025-05/seg…","""http://cgallan.blogspot.com/20…","""/2008/09/ever-catch-yourself-s…","""blogspot.com"""
"""com""",301,"""crawl-data/CC-MAIN-2025-05/seg…","""http://cgallan.blogspot.com/20…","""/2016/06/unfinished-tales-and-…","""blogspot.com"""
"""com""",301,"""crawl-data/CC-MAIN-2025-05/seg…","""http://cgallan.blogspot.com/20…","""/2017/02/an-even-better-way-to…","""blogspot.com"""
"""com""",404,"""crawl-data/CC-MAIN-2025-05/seg…","""https://cgallan.blogspot.com/2…","""/2017/02/an-even-better-way-to…","""blogspot.com"""
"""com""",404,"""crawl-data/CC-MAIN-2025-05/seg…","""https://cgindia.blogspot.com/""","""/""","""blogspot.com"""
…,…,…,…,…,…
"""com""",404,"""crawl-data/CC-MAIN-2025-05/seg…","""https://cgtcecrops.blogspot.co…","""/""","""blogspot.com"""
"""com""",404,"""crawl-data/CC-MAIN-2025-05/seg…","""https://cgulblogger.blogspot.c…","""/2024/09/113-welcome-to-databa…","""blogspot.com"""
"""com""",404,"""crawl-data/CC-MAIN-2025-05/seg…","""https://cgulblogger.blogspot.c…","""/2024/09/ace2024109.html""","""blogspot.com"""


In [44]:
table.schema

Schema([('url_surtkey', String),
        ('url', String),
        ('url_host_name', String),
        ('url_host_tld', String),
        ('url_host_2nd_last_part', String),
        ('url_host_3rd_last_part', String),
        ('url_host_4th_last_part', String),
        ('url_host_5th_last_part', String),
        ('url_host_registry_suffix', String),
        ('url_host_registered_domain', String),
        ('url_host_private_suffix', String),
        ('url_host_private_domain', String),
        ('url_host_name_reversed', String),
        ('url_protocol', String),
        ('url_port', Int32),
        ('url_path', String),
        ('url_query', String),
        ('fetch_time', Datetime(time_unit='ms', time_zone='UTC')),
        ('fetch_status', Int16),
        ('fetch_redirect', String),
        ('content_digest', String),
        ('content_mime_type', String),
        ('content_mime_detected', String),
        ('content_charset', String),
        ('content_languages', String),
        ('conte